In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
from urllib import parse
import numpy as np

네이버 영화 메인 페이지 1~10위 영화 상세페이지 주소 가져오기

In [2]:
def get_detail_url() :
    # 접속할 페이지의 주소
    site = 'https://movie.naver.com/movie/running/current.nhn?order=reserve'

    # 접속한다.
    response = requests.get(site)   

    bs = BeautifulSoup(response.content, 'html.parser')

    # a 태그들을 가져온다.
    a_list = bs.select('.top_thumb_lst a')

    # href 속성을 가져온다.
    df = pd.DataFrame()
    for idx in range(10) :       # 상위 10개만 가져오기
        href = a_list[idx].get('href')
        
        # 가져온 href 속성의 주소를 분석한 객체를 생성한다.
        a1 = parse.urlparse(href)
        
        # 주소를 분석한 객체서 쿼리 스트링을 가져온다
        query_str = parse.parse_qs(a1.query)
        
        # 추출한 쿼리스트링 데이터에서 원하는 파라미터 데이터를 추출한다.
        code = query_str['code'][0]
        
        df = df.append([[code]], ignore_index=True)

    df.columns = ['code']
    df.to_csv('movie_code_list.csv', index=False, encoding='utf-8-sig')
    print('주소 저장 완료')

In [3]:
get_detail_url()

주소 저장 완료


평점 더보기 주소 스크랩

In [4]:
def get_reple_href() :
    # 스크랩한 영화 코드를 불러온다.
    code_frame = pd.read_csv('movie_code_list.csv')
    code_list = code_frame['code'].tolist()

    # 테스트용
 #   code_list = ['156464', '109906']

    # 영화코드와 주소를 합쳐서 요청할 주소를 만든다.
    url_list = pd.DataFrame()
    for code in code_list:
        site = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%s&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false' % code
        # print(site)
        url_list = url_list.append([[site]], ignore_index=True)

    url_list.columns = ['url']
    url_list.to_csv('movie_url_list.csv', index=False, encoding='utf-8-sig')
    print('저장완료')

In [5]:
get_reple_href()

저장완료


예매순 1~10 영화의 1페이지부터 마지막페이지 까지 순회하면서 평점과 리뷰 데이터 가져와서 저장

In [6]:
def get_reply_data() :
    # csv에 저장되어 있는 url 데이터를 가져온다.
    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    # for url in url_list :
    #    print(url)
    # url = url_list[0]

    for url in url_list :
        print(url)
        # 해당 영화의 첫 페이지 html 데이터를 가져온다. (총 몇건의 리뷰가 있는지 확인해서 "페이지수"를 계산하기 위함)
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'html.parser')
        # print(bs)

        # 총 페이지 수를 구한다.
        strong = bs.select('.total em')
        #1->0
        score_total = int(strong[0].text.replace(',', ''))    # 쉼표 없애기 / int(정수형)로 만들기
        # print(score_total)
        pageCnt = score_total // 10     # 한페이지당 10개의 리뷰가 있어서
        if score_total % 10 > 0 :
            pageCnt += 1
        print(pageCnt)

        # 전체 페이지를 돌면서 140평 데이터를 가져온다.
        # 현재 페이지
        now_page = 1

        #pageCnt = 5        # 테스트로 5페이지까지만
        
        while now_page <= pageCnt :
            #sleep(1)
            #sleep(0.0001)
            # 요청할 페이지의 주소
            url2 = url + '&page=' + str(now_page)
            # print(url2)

            # 140자평 데이터를 추출한다.
            response2 = requests.get(url2)
            bs2 = BeautifulSoup(response2.content, 'html.parser')

            result_df = pd.DataFrame()

            # li 태그들을 가져온다.(score_reple 태그-리뷰-를 포함하고 있는)
            lis = bs2.select('.score_result li')

            for obj in lis :
                # 평점
                star_score = obj.select('.star_score em')[0].text
                # 140자평
                score_reple = obj.select('.score_reple p')[0].text
                # print(star_score)
                # print(score_reple)

                # 저장한다.
                result_df = result_df.append([[score_reple, star_score]], ignore_index=True)

            if os.path.exists('star_score.csv') == False :     # 아직 파일이 없으면 파일을 만든다.
                result_df.columns = ['text', 'score']
                result_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig')
            else :                                               # 이미 파일이 있으면 결과를 더한다.
                result_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

            print("%d / %d" % (now_page, pageCnt))               # 진행경과를 보여준다. n번째 중 몇 번째 진행중인지.
            now_page += 1

    print('저장완료')

In [7]:
get_reply_data()

https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=191633&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
41
1 / 41
2 / 41
3 / 41
4 / 41
5 / 41
6 / 41
7 / 41
8 / 41
9 / 41
10 / 41
11 / 41
12 / 41
13 / 41
14 / 41
15 / 41
16 / 41
17 / 41
18 / 41
19 / 41
20 / 41
21 / 41
22 / 41
23 / 41
24 / 41
25 / 41
26 / 41
27 / 41
28 / 41
29 / 41
30 / 41
31 / 41
32 / 41
33 / 41
34 / 41
35 / 41
36 / 41
37 / 41
38 / 41
39 / 41
40 / 41
41 / 41
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=152691&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
26
1 / 26
2 / 26
3 / 26
4 / 26
5 / 26
6 / 26
7 / 26
8 / 26
9 / 26
10 / 26
11 / 26
12 / 26
13 / 26
14 / 26
15 / 26
16 / 26
17 / 26
18 / 26
19 / 26
20 / 26
21 / 26
22 / 26
23 / 26
24 / 26
25 / 26
26 / 26
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=14448&type=after&isActualPointWriteExecute=fal

315 / 519
316 / 519
317 / 519
318 / 519
319 / 519
320 / 519
321 / 519
322 / 519
323 / 519
324 / 519
325 / 519
326 / 519
327 / 519
328 / 519
329 / 519
330 / 519
331 / 519
332 / 519
333 / 519
334 / 519
335 / 519
336 / 519
337 / 519
338 / 519
339 / 519
340 / 519
341 / 519
342 / 519
343 / 519
344 / 519
345 / 519
346 / 519
347 / 519
348 / 519
349 / 519
350 / 519
351 / 519
352 / 519
353 / 519
354 / 519
355 / 519
356 / 519
357 / 519
358 / 519
359 / 519
360 / 519
361 / 519
362 / 519
363 / 519
364 / 519
365 / 519
366 / 519
367 / 519
368 / 519
369 / 519
370 / 519
371 / 519
372 / 519
373 / 519
374 / 519
375 / 519
376 / 519
377 / 519
378 / 519
379 / 519
380 / 519
381 / 519
382 / 519
383 / 519
384 / 519
385 / 519
386 / 519
387 / 519
388 / 519
389 / 519
390 / 519
391 / 519
392 / 519
393 / 519
394 / 519
395 / 519
396 / 519
397 / 519
398 / 519
399 / 519
400 / 519
401 / 519
402 / 519
403 / 519
404 / 519
405 / 519
406 / 519
407 / 519
408 / 519
409 / 519
410 / 519
411 / 519
412 / 519
413 / 519
414 / 519


510 / 2323
511 / 2323
512 / 2323
513 / 2323
514 / 2323
515 / 2323
516 / 2323
517 / 2323
518 / 2323
519 / 2323
520 / 2323
521 / 2323
522 / 2323
523 / 2323
524 / 2323
525 / 2323
526 / 2323
527 / 2323
528 / 2323
529 / 2323
530 / 2323
531 / 2323
532 / 2323
533 / 2323
534 / 2323
535 / 2323
536 / 2323
537 / 2323
538 / 2323
539 / 2323
540 / 2323
541 / 2323
542 / 2323
543 / 2323
544 / 2323
545 / 2323
546 / 2323
547 / 2323
548 / 2323
549 / 2323
550 / 2323
551 / 2323
552 / 2323
553 / 2323
554 / 2323
555 / 2323
556 / 2323
557 / 2323
558 / 2323
559 / 2323
560 / 2323
561 / 2323
562 / 2323
563 / 2323
564 / 2323
565 / 2323
566 / 2323
567 / 2323
568 / 2323
569 / 2323
570 / 2323
571 / 2323
572 / 2323
573 / 2323
574 / 2323
575 / 2323
576 / 2323
577 / 2323
578 / 2323
579 / 2323
580 / 2323
581 / 2323
582 / 2323
583 / 2323
584 / 2323
585 / 2323
586 / 2323
587 / 2323
588 / 2323
589 / 2323
590 / 2323
591 / 2323
592 / 2323
593 / 2323
594 / 2323
595 / 2323
596 / 2323
597 / 2323
598 / 2323
599 / 2323
600 / 2323

1235 / 2323
1236 / 2323
1237 / 2323
1238 / 2323
1239 / 2323
1240 / 2323
1241 / 2323
1242 / 2323
1243 / 2323
1244 / 2323
1245 / 2323
1246 / 2323
1247 / 2323
1248 / 2323
1249 / 2323
1250 / 2323
1251 / 2323
1252 / 2323
1253 / 2323
1254 / 2323
1255 / 2323
1256 / 2323
1257 / 2323
1258 / 2323
1259 / 2323
1260 / 2323
1261 / 2323
1262 / 2323
1263 / 2323
1264 / 2323
1265 / 2323
1266 / 2323
1267 / 2323
1268 / 2323
1269 / 2323
1270 / 2323
1271 / 2323
1272 / 2323
1273 / 2323
1274 / 2323
1275 / 2323
1276 / 2323
1277 / 2323
1278 / 2323
1279 / 2323
1280 / 2323
1281 / 2323
1282 / 2323
1283 / 2323
1284 / 2323
1285 / 2323
1286 / 2323
1287 / 2323
1288 / 2323
1289 / 2323
1290 / 2323
1291 / 2323
1292 / 2323
1293 / 2323
1294 / 2323
1295 / 2323
1296 / 2323
1297 / 2323
1298 / 2323
1299 / 2323
1300 / 2323
1301 / 2323
1302 / 2323
1303 / 2323
1304 / 2323
1305 / 2323
1306 / 2323
1307 / 2323
1308 / 2323
1309 / 2323
1310 / 2323
1311 / 2323
1312 / 2323
1313 / 2323
1314 / 2323
1315 / 2323
1316 / 2323
1317 / 2323
1318

1919 / 2323
1920 / 2323
1921 / 2323
1922 / 2323
1923 / 2323
1924 / 2323
1925 / 2323
1926 / 2323
1927 / 2323
1928 / 2323
1929 / 2323
1930 / 2323
1931 / 2323
1932 / 2323
1933 / 2323
1934 / 2323
1935 / 2323
1936 / 2323
1937 / 2323
1938 / 2323
1939 / 2323
1940 / 2323
1941 / 2323
1942 / 2323
1943 / 2323
1944 / 2323
1945 / 2323
1946 / 2323
1947 / 2323
1948 / 2323
1949 / 2323
1950 / 2323
1951 / 2323
1952 / 2323
1953 / 2323
1954 / 2323
1955 / 2323
1956 / 2323
1957 / 2323
1958 / 2323
1959 / 2323
1960 / 2323
1961 / 2323
1962 / 2323
1963 / 2323
1964 / 2323
1965 / 2323
1966 / 2323
1967 / 2323
1968 / 2323
1969 / 2323
1970 / 2323
1971 / 2323
1972 / 2323
1973 / 2323
1974 / 2323
1975 / 2323
1976 / 2323
1977 / 2323
1978 / 2323
1979 / 2323
1980 / 2323
1981 / 2323
1982 / 2323
1983 / 2323
1984 / 2323
1985 / 2323
1986 / 2323
1987 / 2323
1988 / 2323
1989 / 2323
1990 / 2323
1991 / 2323
1992 / 2323
1993 / 2323
1994 / 2323
1995 / 2323
1996 / 2323
1997 / 2323
1998 / 2323
1999 / 2323
2000 / 2323
2001 / 2323
2002

In [8]:
# 140자평 데이터 전처리 함수
def text_preprocessing(text) :
    if text.startswith('관람객') :
        return text[3:]
    elif text.startswith('스포일러가 포함된 감상평입니다. 감상평 보기'):
        return text[23:]
    else :
        return text

# 평점 전처리 함수
def star_preprocessing(text) :
    value = int(text)

    if value <= 5 :
        return '0'
    else :
        return '1'

In [9]:
def data_preprocessing() :
    # 수집한 데이터를 읽어온다.
    df = pd.read_csv('star_score.csv')

    # 전처리를 수행한다.
    df['text'] = df['text'].apply(text_preprocessing)
    df['score'] = df['score'].apply(star_preprocessing)

    # 학습데이터와 테스트 데이터로 나눈다.
    text_list = df['text'].tolist()
    star_list = df['score'].tolist()

    from sklearn.model_selection import train_test_split

    # 70%는 학습, 30%는 test
    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.3, random_state=0)

    return text_train, text_test, star_train, star_test


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
from konlpy.tag import *

In [10]:
data_preprocessing()

(['\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t올해 들어 본 가장 좋은 영화, 아니 인생 영화 중 하나가 됐다. 전쟁의 허망함.. 그럼에도 해내지 않으면 살아갈 수 없는 군인의 심리가 마치 직접 겪고있는 듯 생생하게 느껴진다. 정말 좋은 영화다. \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n',
  '\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tVOD동시개봉 집에서 볼수 있어서 만족!!!!트롤 들이 다 너무 귀엽고 재밋는 영상에 흥겨운 음악~ 사랑스러운영화에요♡♡ \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n',
  '\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t무조건 보십시오.... \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t

In [11]:
# 형태소 분석을 위한 함수
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)

In [12]:
def learning(X_train, y_train, X_test, y_test):
    # 주어진 데이터를 단어 사전으로 만들고 각 단어의 빈도수를 계산한 후 벡터화 하는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer, ngram_range=(1,2)) #ngram (1,2)
     
    # 문장별 나오는 단어수 세서 수치화, 벡터화해서 학습을 시킨다.
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습한다.
    pipe.fit(X_train, y_train)

    # 학습 정확도 측정
    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))

    # 학습된 모델을 저장한다.
    with open('pipe.dat', 'wb') as fp :
        pickle.dump(pipe, fp)
        
    print('저장완료')

In [14]:
learing()

C:\Users\GiWoong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9644978137440449
저장완료


In [13]:
# 스크래핑 함수
def scrapping() :

    # 각 영화 코드 데이터를 가져와 저장한다.
    get_detail_url()

    # 140자 평 데이터가 있는 페이지의 주소를 저장한다.
    get_reple_href()

    # 140평 데이터를 가져온다.
    get_reply_data()

# 학습 함수
def learing() :
    text_train, text_test, star_train, star_test = data_preprocessing()
    learning(text_train, star_train, text_test, star_test)
